In [1]:
import findspark
findspark.init('/Users/kevinblum/Apache-Spark/spark-3.1.2-bin-hadoop3.2')
import tweepy
import string
import os
import re
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.ml.feature import CountVectorizer
#consumer_key = "Ax79DI6gXdMjOCQyQ61v2Khbj"
#consumer_secret = "DBSmfUL50FlhpVnePa373lOpBFffJjrGInXbSGG4T7Ze2RRQTl"
#access_token = "1418173405988376579-9g3SF5klnWOgd62nUfTfaqrQdzkJRA"
#access_token_secret = "q3jO28xbGDlvE6djCNeoSMQbtkosZYT7mBTv0O6wznMpw"
# Creating the authentication object
#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
#auth.set_access_token(access_token, access_token_secret)
#api = tweepy.API(auth)


In [2]:
#open 2 command prompts
#terminal 1
#cd %SPARK_HOME%
#bin\spark-class2.cmd org.apache.spark.deploy.master.Master
#open in browser localhost:8080
#terminal 2
#cd %SPARK_HOME%
#bin\spark-class2.cmd org.apache.spark.deploy.worker.Worker -c 4 -m 32G spark://192.168.1.227:7077
#open in browser localhost:8081
spark = SparkSession.builder.master('spark://Kevins-MBP.home:7077').appName('classifier').config('spark.executor.memory', '32g').getOrCreate()
df = spark.read.options(delimiter=",",header=False).csv("trainingandtestdata/training.1600000.processed.noemoticon.csv")
df=df.toDF('polarity','tweet_id','tweet_date','query_string','tweet_screen_name','Tweet')
df2=spark.read.options(delimiter=",",header=False).csv("trainingandtestdata/testdata.manual.2009.06.14.csv")
df2=df2.toDF('polarity','tweet_id','tweet_date','query_string','tweet_screen_name','Tweet')

In [ ]:
from pyspark.sql.types import StringType
def preprocessing(text):
    # process the tweets

    #Convert to lower case
    tweet = text.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+',' ',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet
tem=df.union(df2)
brand_udf=udf(preprocessing,StringType())
tem=tem.withColumn('text',brand_udf(tem['Tweet']))

In [ ]:
from pyspark.sql.types import IntegerType
tem1=tem.select('tweet_id','Tweet','text','polarity')
tem1 = tem1.withColumn('polarity', tem1['polarity'].cast(IntegerType()))
(trainingData, testData) = tem1.randomSplit([0.8, 0.2])

In [5]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer

# Configure an ML pipeline, which consists of tree stages: tokenizer, hashingTF, and nb.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="rawFeatures")
idf = IDF(minDocFreq=2, inputCol="rawFeatures", outputCol="features")

lr = LogisticRegression(labelCol='polarity')

# Pipeline Architecture
pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, lr])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

In [6]:
predictions = model.transform(testData)
predictions1=predictions.toPandas()
# Select example rows to display.
#predictions.select("text", "polarity", "prediction").show(5,False)

In [8]:
predictions.head()

Row(tweet_id='1467810369', Tweet="@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", text=" - awww, that's a bummer. you shoulda got david carr of third day to do it. ;d", polarity=0, words=['', '-', 'awww,', "that's", 'a', 'bummer.', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it.', ';d'], rawFeatures=SparseVector(262144, {27576: 1.0, 38640: 1.0, 52803: 1.0, 107107: 1.0, 115994: 1.0, 127420: 1.0, 143501: 1.0, 148101: 1.0, 168211: 1.0, 173018: 1.0, 196041: 1.0, 214962: 1.0, 219087: 1.0, 225898: 1.0, 226790: 1.0, 242377: 1.0, 249180: 1.0, 254061: 1.0}), features=SparseVector(262144, {27576: 1.2567, 38640: 3.2717, 52803: 8.4601, 107107: 1.58, 115994: 3.3472, 127420: 8.517, 143501: 4.6982, 148101: 4.3451, 168211: 7.1534, 173018: 6.7644, 196041: 7.7009, 214962: 2.078, 219087: 2.2437, 225898: 3.305, 226790: 10.0549, 242377: 7.8141, 249180: 0.8098, 254061: 3.3144}), rawPrediction=DenseVector([

In [9]:
testData.show(5)

+----------+--------------------+--------------------+--------+
|  tweet_id|               Tweet|                text|polarity|
+----------+--------------------+--------------------+--------+
|1467810369|@switchfoot http:...| - awww, that's a...|       0|
|1467811372|@Kwesidei not the...| not the whole crew |       0|
|1467812784|@smarrison i woul...| i would've been ...|       0|
|1467814192|blagh class at 8 ...|blagh class at 8 ...|       0|
|1467815923|some1 hacked my a...|some1 hacked my a...|       0|
+----------+--------------------+--------------------+--------+
only showing top 5 rows



In [11]:
testData.printSchema()

root
 |-- tweet_id: string (nullable = true)
 |-- Tweet: string (nullable = true)
 |-- text: string (nullable = true)
 |-- polarity: integer (nullable = true)



In [12]:
testData

DataFrame[tweet_id: string, Tweet: string, text: string, polarity: int]

In [10]:
model.save("/Users/kevinblum/BigDataProj/SparkTwitterStream/models/lr")

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="polarity", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(accuracy)

In [ ]:
import pandas as pd
import json
testData=testData.toPandas()
predictions=predictions1
testData['prediction']=predictions['prediction']
testData=testData[['tweet_id','Tweet','polarity','prediction']]
spark.stop()

In [ ]:
import pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase"]
mycol = mydb["tweet"]
records = json.loads(testData.T.to_json()).values()
mydb.mycol.insert_many(records)

In [ ]:
print(lr)